In [1]:
import Enum_data as ed
import general_functions as gf
import operational_functions as of
import monthly_report_functions as mf
import income_statement_functions as isf
import others_functions as otf
import overhead_functions as ovf
import quantity_turnover_functions as qf
import raw_material_functions as rf
import cogs_functions as cf
import Cost_functions as cof

In [2]:
def run_program(symbol: str, year: int = 1400):
    report_types = [value.value for value in ed.names]
    all_reports = {}

    financial_years=gf.get_financial_years(symbol, year)
    for year in financial_years:
        parse_date = gf.parse_date_persian(year)
        dummy_dict = {}
        for report in report_types:
            report_type = ed.names[report].value
            sheet_num = ed.sheets[report_type].value
            table = ed.tabels[report_type].value
            reports = gf.get_results(symbol, parse_date, report_type, sheet_num)
            dummy_dict[report_type] = reports
        all_reports[year] = dummy_dict
    return all_reports

In [3]:
symbol = 'فخوز'
all_reports = run_program(symbol)

In [5]:
#Excel Format data
filtered_all_reports = {}
for fin_year in sorted(list(all_reports.keys()), reverse=True):
    filtered_all_reports[fin_year] = {}
    for report_type in [value.value for value in ed.names]:
        if report_type in ['Monthly_report']: continue
        if fin_year in all_reports.keys():
            if all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]['period'] == 12:
                filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]
            else:
                filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]
filtered_all_reports

{'14041229': {'Operational': {'period': '6',
   'publish': 14040824,
   'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۶ ماهه منتهی به  ۱۴۰۴/۰۶/۳۱ (حسابرسی نشده)(اصلاحیه)',
   'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
  'COGS': {'period': '6',
   'publish': 14040824,
   'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۶ ماهه منتهی به  ۱۴۰۴/۰۶/۳۱ (حسابرسی نشده)(اصلاحیه)',
   'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
  'Quantity_Turnover': {'period': '6',
   'publish': 14040824,
   'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۶ ماهه منتهی به  ۱۴۰۴/۰۶/۳۱ (حسابرسی نشده)(اصلاحیه)',
   'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=21'},
  'RawMaterial': {'period': '6',
   'publish': 14040824,
   'title': 'اطلاعات

In [6]:
import re
def extract_date(title: str) -> int:
    trans = str.maketrans("۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩", "01234567890123456789")
    t = title.translate(trans)

    m = re.search(r"\b(1[0-9]{3}|0[0-9]{3})/([01]?\d)/([0-3]?\d)\b", t)
    date = m.group(0) if m else None
    date = date.replace('/',"")
    return int(date)

In [18]:
def sort_df_dic(all_data: dict, report_type: str, dummy_dic: dict) -> dict:
    if report_type not in all_data:
        all_data[report_type] = {}

    for _, df in dummy_dic.items():
        if df is None:
            continue

        dt = df["Date"][0]

        if dt not in all_data[report_type]:
            all_data[report_type][dt] = df

    
    return all_data

In [19]:
all_data = {}
for year, report_types in filtered_all_reports.items():
    for report_type, st_dic in report_types.items():
        if report_type == "Monthly_report":
            continue

        date = extract_date(st_dic["title"])
        period = st_dic["period"]
        publish = st_dic["publish"]
        url = st_dic["url"]

        if report_type == 'Operational':
            d = of.create_operational_dataframe(symbol, url, date, period, publish)
            all_data = sort_df_dic(all_data, report_type, d)
        

In [21]:
all_data['Operational'].keys()

dict_keys([14031230, 14040631, 14041229, 14021229, 14011229, 14001229])

In [22]:
all_data = {}
for year, report_types in filtered_all_reports.items():
    for report_type, st_dic in report_types.items(): 
        date = extract_date(st_dic['title'])
        period = st_dic['period']
        publish = st_dic['publish']
        url = st_dic['url']

        if report_type == 'Monthly_report' :
            #mf.create_Monthly_report_dataframe(symbol, url, date, publish)
            continue
        if report_type == 'Operational':
            #d = of.create_operational_dataframe(symbol, url, date, period, publish)
            #all_data = sort_df_dic(all_data, report_type, d)
            continue
        if report_type == 'COGS' :
            #print(cf.create_cogs_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Cost' :
            #print(cof.creat_Cost_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Incoeme_Statment' :
            #print(isf.create_Incoeme_Statment_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Others' :
            #print(otf.create_Others_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Overhead' :
            #print(ovf.creat_Overhead_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'RawMaterial' :
            break
            #print(rf.creat_raw_material_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Quantity_Turnover' :
            #print(qf.create_Quantity_Turnover_dataframe(symbol, url, date, period, publish))
            continue

In [23]:
symbol, url, date, period, publish

('فخوز',
 'https://codal.ir/Reports/Decision.aspx?LetterSerial=XvDHnRFUahZP67qCeNrhag%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=21',
 14011229,
 12,
 14020418)

In [26]:
import requests
import json
import re
import polars as pl
import Enum_data as ed
import general_functions as gf
from persiantools import characters, digits

In [116]:
def create_dict_dataframes(url: str, date: int, report_type: str) -> dict:
    all_data ={ 'report_this_year': pl.DataFrame(),
                'est_remain': pl.DataFrame(),
                'est_next_year': pl.DataFrame()}

    cells_tuples = rf.get_table(url, ed.tabels[report_type].value)

    dates = sorted(list(set([i[-1] for i in cells_tuples if i[-1] != ''])))
    for date_ in dates:
        products = []
        filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == date_]
        products = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '']
        df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
        df = df.pivot(values="value", on="col", index="row").sort("row")
        
        df_products = pl.from_records(products, schema=["col", "row", "value"], orient="row")
        df_products = df_products.pivot(values="value", on="col", index="row").sort("row")

        df_products = df_products.filter(df_products['row']>=df['row'][0] ,df_products['row']<=df['row'][-1])
        df = df_products.join(df, on="row", how="left")

        for i, col in enumerate(df.columns):
            cols = ed.cols[report_type].value
            value = df[col][0]        # value from first row
            if(isinstance(value, str)):
                for val in value.split():
                    if'/' in val:
                        num = int(val.replace('/', ''))
                        next_cols = df.columns[i : i + cols + 1]
                        cols_to_select = ['1','2', *next_cols]
                        
                        if num == date :
                            all_data['report_this_year'] = df.select(cols_to_select)
                            all_data['report_this_year'] = all_data['report_this_year'].with_row_index("row")
                            all_data['report_this_year'] = all_data['report_this_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                        else:
                            df = df.with_columns(pl.col("1").map_elements(lambda x: characters.ar_to_fa(x)).alias("1"))
                            split_row = (df.filter(pl.col("1") == 'جمع کـل').select(pl.col("row").first()).item())
                            est_df_remain = df.filter(pl.col("row") <= split_row)
                            est_next_year  = df.filter(pl.col("row") > split_row)

                            remain_year = est_df_remain.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()
                            next_yesr_year = est_next_year.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()
                            
                            if remain_year == next_yesr_year:
                                    all_data['est_next_year'] = est_df_remain
                                    all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                            else:
                                    all_data['est_remain'] = est_df_remain
                                    all_data['est_remain'] = all_data['est_remain'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))                          

                                    all_data['est_next_year'] = est_next_year
                                    all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
    
    for key, value in all_data.items():
        if(value.is_empty()):
            all_data[key] = None
    return all_data

In [118]:
def creat_raw_material_dataframe(symbol: str, url: str, date: int, period: int, publish: int) -> dict:
    all_data = create_dict_dataframes(url, date, 'RawMaterial')

    for key, data in all_data.items():
        if data is None: continue      
        cols = data.columns
        sorted_cols = sorted(cols, key=lambda x: (0 if x == "row" else 1, int(x) if x.isdigit() else float("inf")))
        sorted_cols = ['row', 'Date', '1', '2', '3', '4', '6', '7', '9', '10', '12', '13']
        data = data.select(sorted_cols)

        col_name = {
            "Raw Material": '1',
            'Unit': '2',
            "Beginning Inventory_qn": '3',
            "Beginning Inventory_pr": '4',
            "Purchases During the Period_qn": '6',
            "Purchases During the Period_pr": '7',
            "Consumption_qn": '9',
            "Consumption_pr": '10',
            "Ending Inventory_qn": '12',
            "Ending Inventory_pr": '13'
        }
        
        data = data.with_columns(data["1"].map_elements(lambda x: characters.ar_to_fa(x)).alias('1'))
        idx_internal_raw = data.filter(data['1'] == "مواد اولیه داخلی:")['row'][0]
        idx_total_internal_raw= data.filter(data['1']== "جمع مواد اولیه داخلی")['row'][0]
        df_internal_raw = data.filter(data['row']>idx_internal_raw,data['row']<idx_total_internal_raw-1)
        df_internal_raw=df_internal_raw.with_columns(pl.lit("داخلی").alias("Type"))
        idx_export_raw = data.filter(data['1'] == "مواد اولیه وارداتی:")['row'][0]
        idx_total_export_raw= data.filter(data['1']== "جمع مواد اولیه وارداتی")['row'][0]
        df_export_raw = data.filter(data['row']>idx_export_raw,data['row']<idx_total_export_raw-1)
        df_export_raw=df_export_raw.with_columns(pl.lit("وارداتی").alias("Type"))
        df_goods = pl.concat([df_internal_raw,df_export_raw])

        rename_map = {v: k for k, v in col_name.items()}
        df_goods = df_goods.rename(rename_map)
        
        df_goods = df_goods.insert_column(0, pl.lit(symbol).alias("Symbol"))
        df_goods = df_goods.insert_column(2, pl.lit(period).alias("Period"))
        df_goods = df_goods.insert_column(3, pl.lit(publish).alias("Publish"))
        cs = ['Symbol','Period','Publish','Date', 'Raw Material', 'Unit', 'Type', 'Beginning Inventory_qn', 'Beginning Inventory_pr', 'Purchases During the Period_qn', 'Purchases During the Period_pr', 'Consumption_qn', 'Consumption_pr', 'Ending Inventory_qn', 'Ending Inventory_pr']
        df_goods = df_goods[cs]
        all_data[key] = df_goods
    
    return all_data     

In [119]:
creat_raw_material_dataframe(symbol, url, date, period, publish)


{'report_this_year': shape: (7, 15)
 ┌────────┬────────┬──────────┬──────────┬───┬─────────────┬─────────────┬─────────────┬────────────┐
 │ Symbol ┆ Period ┆ Publish  ┆ Date     ┆ … ┆ Consumption ┆ Consumption ┆ Ending Inve ┆ Ending Inv │
 │ ---    ┆ ---    ┆ ---      ┆ ---      ┆   ┆ _qn         ┆ _pr         ┆ ntory_qn    ┆ entory_pr  │
 │ str    ┆ i32    ┆ i32      ┆ i32      ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---        │
 │        ┆        ┆          ┆          ┆   ┆ str         ┆ str         ┆ str         ┆ str        │
 ╞════════╪════════╪══════════╪══════════╪═══╪═════════════╪═════════════╪═════════════╪════════════╡
 │ فخوز   ┆ 12     ┆ 14020418 ┆ 14011229 ┆ … ┆ 4950935     ┆ 33772318    ┆ 1566757     ┆ 37366333   │
 │ فخوز   ┆ 12     ┆ 14020418 ┆ 14011229 ┆ … ┆ 232678      ┆ 117846565   ┆ 22795       ┆ 134067208  │
 │ فخوز   ┆ 12     ┆ 14020418 ┆ 14011229 ┆ … ┆ 606047      ┆ 92984495    ┆ 61753       ┆ 103415138  │
 │ فخوز   ┆ 12     ┆ 14020418 ┆ 14011229 ┆ … ┆

In [110]:
all_data ={ 'report_this_year': pl.DataFrame(),
            'est_remain': pl.DataFrame(),
            'est_next_year': pl.DataFrame()}

for date_ in dates:
    
    visited = False
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '' or i[-1] == (date_)]
    products = []
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == date_]
    products = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '']
    df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df = df.pivot(values="value", on="col", index="row").sort("row")
    
    
    df_products = pl.from_records(products, schema=["col", "row", "value"], orient="row")
    df_products = df_products.pivot(values="value", on="col", index="row").sort("row")

    
    df_products = df_products.filter(df_products['row']>=df['row'][0] ,df_products['row']<=df['row'][-1])
    df = df_products.join(df, on="row", how="left")

    for i, col in enumerate(df.columns):
        cols = ed.cols[report_type].value
        value = df[col][0]        # value from first row
        if(isinstance(value, str)):
            for val in value.split():
                if'/' in val:
                    num = int(val.replace('/', ''))
                    next_cols = df.columns[i : i + cols + 1]
                    cols_to_select = ['1','2', *next_cols]

                    if num == date :
                        all_data['report_this_year'] = df.select(cols_to_select)
                        all_data['report_this_year'] = all_data['report_this_year'].with_row_index("row")
                        all_data['report_this_year'] = all_data['report_this_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                    else:
                        df = df.with_columns(pl.col("1").map_elements(lambda x: characters.ar_to_fa(x)).alias("1"))
                        split_row = (df.filter(pl.col("1") == 'جمع کـل').select(pl.col("row").first()).item())
                        est_df_remain = df.filter(pl.col("row") <= split_row)
                        est_next_year  = df.filter(pl.col("row") > split_row)

                        remain_year = est_df_remain.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()
                        next_yesr_year = est_next_year.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()
                        
                        if remain_year == next_yesr_year:
                                all_data['est_next_year'] = est_df_remain
                                all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                        else:
                                all_data['est_remain'] = est_df_remain
                                all_data['est_remain'] = all_data['est_remain'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))                          

                                all_data['est_next_year'] = est_next_year
                                all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
all_data['est_next_year']

row,1,2,3,4,5,6,7,8,9,10,11,12,13,Date,14
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i32,str
45,"""شرح""","""واحد""","""برآورد دوره 12 ماهه منتهی به 1…",null,null,null,null,null,null,null,null,null,null,14021229,null
49,"""مواد اولیه داخلی:""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",14021229,""""""
50,"""سنگ آهن""","""تن""","""1566757""","""37366333""","""58543964""","""6150000""","""45851341""","""281985745""","""6200000""","""45841583""","""284217812""","""1516757""","""37126512""",14021229,"""56311897"""
51,"""آهن قراضه و چدن""","""تن""","""22795""","""134067208""","""3056062""","""260000""","""144948565""","""37686627""","""265000""","""144712234""","""38348742""","""17795""","""134529194""",14021229,"""2393947"""
52,""" اسفنجی خریداری شده""","""تن""","""61753""","""103415138""","""6386195""","""465000""","""128032344""","""59535040""","""511000""","""128032344""","""65424528""","""15753""","""31530946""",14021229,"""496707"""
53,"""فروآلیاژها""","""تن""","""16622""","""535286067""","""8897525""","""72460""","""637996715""","""46229242""","""49960""","""657041753""","""32825806""","""39122""","""570036322""",14021229,"""22300961"""
54,"""مواد افزودنی""","""تن""","""12437""","""215003377""","""2673997""","""137590""","""103323657""","""14216302""","""132606""","""107731656""","""14285864""","""17421""","""149499742""",14021229,"""2604435"""
55,"""گندله خریداری""","""تن""","""59605""","""44985337""","""2681351""","""300000""","""55485533""","""16645660""","""342000""","""55485532""","""18976052""","""17605""","""19935189""",14021229,"""350959"""
56,"""سایرمواد""","""تن""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",14021229,"""0"""


In [91]:
df = df.with_columns(pl.col("1").map_elements(lambda x: characters.ar_to_fa(x)).alias("1"))
split_row = (df.filter(pl.col("1") == 'جمع کـل').select(pl.col("row").first()).item())
est_df_remain = df.filter(pl.col("row") <= split_row)
est_next_year  = df.filter(pl.col("row") > split_row)

In [107]:
remain_year = est_df_remain.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()
next_year = est_next_year.select(pl.col("3").str.extract(r'(\d{4})/\d{2}/\d{2}', 1).cast(pl.Int32))[0].item()

In [108]:
remain_year, next_year

(1402, 1402)

In [83]:
df.filter(pl.col("1").map_elements(lambda x: characters.ar_to_fa(x)).alias('1') == 'جمع کـل')['row']

row
i64
62
73


In [18]:
test = {}
for year, report_dic in all_data.items():
    for name, df in report_dic.items():
        if df is None : continue
        if df['Date'][0] not in test:
            test[df['Date'][0]] = df
test


{14031230: shape: (7, 10)
 ┌────────┬──────────┬────────┬──────────┬───┬─────────┬────────────┬─────────┬───────────┐
 │ Symbol ┆ Date     ┆ Period ┆ Publish  ┆ … ┆ Type    ┆ Production ┆ Sales   ┆ Price     │
 │ ---    ┆ ---      ┆ ---    ┆ ---      ┆   ┆ ---     ┆ ---        ┆ ---     ┆ ---       │
 │ str    ┆ i32      ┆ i32    ┆ i32      ┆   ┆ str     ┆ i64        ┆ i64     ┆ i64       │
 ╞════════╪══════════╪════════╪══════════╪═══╪═════════╪════════════╪═════════╪═══════════╡
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 4445096    ┆ 555165  ┆ 129478816 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 967578     ┆ 858953  ┆ 240694072 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 2050460    ┆ 1329807 ┆ 234611990 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 5327975    ┆ 15661   ┆ 54727157  │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ صادراتی ┆ 0          ┆ 103361  ┆ 262062674 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ صادراتی

In [60]:
dates_desc = sorted(list(all_reports.keys()), reverse=True)

['14041229', '14031230', '14021229', '14011229']

In [67]:
filtered_all_reports = {}
for fin_year in dates_desc:
    if fin_year in all_reports.keys():
        if all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]['period'] == 12:
            filtered_all_reports[fin_year] = all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]
        else:
            filtered_all_reports[fin_year] = all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]
filtered_all_reports

{'14041229': {'period': '6',
  'publish': 14040824,
  'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۶ ماهه منتهی به  ۱۴۰۴/۰۶/۳۱ (حسابرسی نشده)(اصلاحیه)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14031230': {'period': 12,
  'publish': 14040506,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۳/۱۲/۳۰ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=1z105r5OIg1Yv0h84qhcAw%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14021229': {'period': 12,
  'publish': 14030420,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۲/۱۲/۲۹ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=gN7ITfhbee1nWKRrszGMMA%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14011229': {'period': 12,
  'publish': 14020418,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۱/۱۲/۲۹ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.asp

In [57]:
if dates_desc[0] in all_reports.keys():
    print(all_reports[dates_desc[0]]['Operational'].keys())

dict_keys([14040331, 14040631])


In [9]:
data.columns = [str(i) for i in range(data.width)]
data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
data = data.insert_column(1, pl.lit(int(digits.fa_to_en('14030930').replace('/', ''))).alias("Date"))
data = data.insert_column(2, pl.lit(period).alias("Period"))
data = data.insert_column(3, pl.lit(publish).alias("Publish"))
data

Symbol,Date,Period,Publish,0,1,2,3,4,5,6,7
str,i32,i32,i32,str,str,str,str,u32,i64,i64,i64
"""فخوز""",14030930,9,14031020,"""فخوز""","""اسلب""","""تن""","""داخلی""",3,1252457,1031275,202773181
"""فخوز""",14030930,9,14031020,"""فخوز""","""بلوم ، بیلت ""","""تن""","""داخلی""",4,2147637,1494216,202614534
"""فخوز""",14030930,9,14031020,"""فخوز""","""آهن اسفنجی""","""تن""","""داخلی""",5,3537243,215334,120139588
"""فخوز""",14030930,9,14031020,"""فخوز""","""گندله ""","""تن""","""داخلی""",6,5506540,344373,46911941
"""فخوز""",14030930,9,14031020,"""فخوز""","""اسلب""","""تن""","""صادراتی""",10,0,243040,187682558
"""فخوز""",14030930,9,14031020,"""فخوز""","""بلوم ، بیلت ""","""تن""","""صادراتی""",11,0,718167,191041588
"""فخوز""",14030930,9,14031020,"""فخوز""","""آهن اسفنجی""","""تن""","""صادراتی""",12,0,31960,101468179


In [10]:
symbol = 'فخوز'
period = 9
publish = 14031020
url = 'https://codal.ir/Reports/InterimStatement.aspx?LetterSerial=YQQQaQQQr9ls8uXPQkIRgPeupRng%3d%3d&SheetId=20'
date = 14030930

dict_df = of.create_operational_dataframe(symbol, url, date, period, publish)
df = dict_df['report_this_year']


In [3]:
df

Symbol,Date,Period,Publish,Product,Unit,Type,Production,Sales,Price
str,i32,i32,i32,str,str,str,i64,i64,i64
"""فخوز""",14030930,9,14031020,"""اسلب""","""تن""","""داخلی""",690087,590616,232053901
"""فخوز""",14030930,9,14031020,"""بلوم ، بیلت ""","""تن""","""داخلی""",1530912,984308,224416890
"""فخوز""",14030930,9,14031020,"""آهن اسفنجی""","""تن""","""داخلی""",3247286,500102,125104491
"""فخوز""",14030930,9,14031020,"""گندله ""","""تن""","""داخلی""",4123197,15661,54727284
"""فخوز""",14030930,9,14031020,"""اسلب""","""تن""","""صادراتی""",0,73346,259861533
"""فخوز""",14030930,9,14031020,"""بلوم ، بیلت ""","""تن""","""صادراتی""",0,473525,219169243
"""فخوز""",14030930,9,14031020,"""آهن اسفنجی""","""تن""","""صادراتی""",0,29390,132843688


In [4]:
df.to_pandas().to_excel(
    "output.xlsx",
    sheet_name="Sheet1",
    index=False
)

In [20]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

out = "output.xlsx"

# --- 1) Create a real workbook with a visible sheet (prevents the error) ---
wb = Workbook()
ws = wb.active
ws.title = "Sheet1"
ws.sheet_state = "visible"
ws.sheet_view.rightToLeft = True
wb.save(out)

titles = ["تولید", "فروش", "نرخ"]
tables = [t1, t2, t3]
gap = 2
excel_row = 1  # 1-based Excel row

with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    ws = writer.book["Sheet1"]

    for title, table in zip(titles, tables):
        date_row = table.select("Date").to_pandas().iloc[0].tolist()
        pdf = table.drop("Date").to_pandas()

        ncols = pdf.shape[1]
        last_col = get_column_letter(ncols)

        # title (merged)
        ws.merge_cells(f"A{excel_row}:{last_col}{excel_row}")
        ws[f"A{excel_row}"].value = title
        ws[f"A{excel_row}"].font = Font(bold=True)
        ws[f"A{excel_row}"].alignment = Alignment(horizontal="center")

        # date row (between title and header)
        for i, v in enumerate(date_row, start=1):
            ws.cell(row=excel_row + 1, column=i, value=v)

        # table (header + data) under date row
        pdf.to_excel(writer, sheet_name="Sheet1", index=False, startrow=excel_row + 1)

        # advance: title + date + header + data + gap
        excel_row += 3 + len(pdf) + gap
